# Signal processing course 2018/2019-1 @ ELTE
# Assignment 2
## 10.01.2018

## Task 3

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
# Initialize seaborn with custom settings
# Facecolor values from S. Conradi @S_Conradi/@profConradi
custom_settings = {
    'figure.facecolor': '#f4f0e8',
    'axes.facecolor': '#f4f0e8',
    'axes.edgecolor': '0.7',
    'axes.linewidth' : '2',
    'grid.color': '0.7',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
}
sns.set_theme(palette=sns.color_palette('deep', as_cmap=False),
              rc=custom_settings)

### Calculate Fourier coefficients

In [ ]:
from scipy.integrate import quad

In [ ]:
def fourier_coeffs(f, T, N, t0=0, *args):
    '''
    Calculates the Fourier coefficients :math:`a_k`, :math:`b_k` up to
    an arbitrary order :math:`N` for a given periodic signal using
    numerical integration.

    Parameters
    ----------
    f : callable
        The target periodic function
    T : float
        Period of the function
    N : int
        The highest order of harmonics to be considered in the
        approximation
    t0 : float
        The starting point of the period (default is 0)
    args : tuple, optional
        Extra arguments to pass to `f` inside the lambda func. Should
        not be confused with the optional `args` argument of
        `scipy.integrate.quad()` inside the decorator functions.
    '''
    # Employ the substitution defined above
    w0 = 2*np.pi / T

    def a_k(k):
        ''' Function to integrate for a_k '''
        func = lambda t: f(t, *args) * np.cos(k * t * w0)
        return quad(func, t0, t0 + T)[0] * 2 / T

    def b_k(k):
        ''' Function to integrate for b_k '''
        func = lambda t: f(t, *args) * np.sin(k * t * w0)
        return quad(func, t0, t0 + T)[0] * 2 / T

    a = np.array([a_k(k) for k in range(N)])
    b = np.array([b_k(k) for k in range(N)])  # note: `b_0` is always 0
    return a, b

### Calculate Fourier series partial sum

In [ ]:
def fourier_sin(t, a, b, N, T, fs=128):
    '''
    Approximates a target periodic :math:`f_N(t)` function using the
    sine-cosine form of the Fourier series in the interval
    :math:`[t_0, t_0+T]`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    N : int
        The highest order of harmonics to be considered in the
        approximation
    T : float
        Period of the evaluated function
    fs : float
        Sampling frequency of the evaluated function
    '''
    # Do not enable larger values for N than the number of values in the
    # input `a` or `b` arrays. Also prevent N < 1 values.
    assert (N >= 1) | (len(a) >= 1), 'N should be larger than or equal to 1!'
    N = np.min((N, len(a)))

    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    # Employ the substitution P(t) = w0 * t
    P = 2*np.pi / T * t
    # Calculate the approximation up to an order N
    fN = np.zeros_like(t, dtype=float) + a[0]/2
    for k in range(1, N):
        fN += a[k] * np.cos(P * k) + b[k] * np.sin(P * k)

    if FCHECK:
        return fN[0]
    return fN

In [ ]:
def c_k(k, a, b):
    '''
    Calculates the conditional value of :math:`c_k` from its definition.

    Parameters
    ----------
    k : int
        The respective value of the :math:`k` running index
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    '''
    if k > 0:
        return 0.5 * (a[k] - 1j * b[k])
    elif k == 0:
        return 0.5 * a[0]
    elif k < 0:
        return 0.5 * (a[k] + 1j * b[k])

In [ ]:
def fourier_exp(t, a, b, N, T, fs=128):
    '''
    Approximates a target periodic :math:`f_N(t)` function using the
    exponential form of the Fourier series in the interval
    :math:`[t_0, t_0+T]`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    N : int
        The highest order of harmonics to be considered in the
        approximation
    T : float
        Period of the evaluated function
    fs : float
        Sampling frequency of the evaluated function
    '''
    # Do not enable larger values for N than the number of values in the
    # input `a` or `b` arrays. Also prevent N < 1 values.
    assert (N >= 1) | (len(a) >= 1), 'N should be larger than or equal to 1!'
    N = np.min((N, len(a)))

    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    # Employ the substitution P(t) = i * w0 * t
    P = 1j * 2*np.pi / T * t
    # Sum over 2N+1 number of `k` indices, running between ]-N -> N[
    fN = np.zeros_like(t, dtype=np.complex128)
    for k in range(-N+1, N):
        fN += c_k(k, a, b) * np.exp(P * k)

    if FCHECK:
        return fN[0]
    return fN

### Testing Fourier series on various functions
#### Common hyperparameters

In [ ]:
fs = 256  # Sampling rate for signals
N = 50    # Highest order of harmonics considered

#### 1) Square wave

The definition of a periodic square wave signal with $\tau$ bandwidth, $T$ period, $A$ amplitude and $\phi$ phase shift is
$$
    f(t)
    =
    \begin{cases}
        A, & \left| (t - \phi) \bmod T \right| \leq \frac{\tau}{2} \\
        0, & \left| (t - \phi) \bmod T \right| > \frac{\tau}{2}
    \end{cases}
    \qquad
    t \in \left] -\frac{T}{2}, \frac{T}{2} \right].
$$

In [ ]:
def f1(t, tau, T, phi=0):
    '''
    Generate a square wave signal with a bandwidth of :math:`\tau`, a
    period of :math:`T`, an amplitude of ::math:`1` and a phase shift
    of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    tau : float
        Bandwidth of the generated square wave signal
    T : float
        Period of the output signal
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T/2)%T - T/2  # Apply periodicity and phase shift
    x = np.abs(t)
    mask = (x <= tau/2)          # Apply definition of square wave
    x[mask] = 1                  # t - phi <= tau/2 --> A = 1
    x[~mask] = 0                 # t - phi >  tau/2 --> A = 0

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
tau = np.pi  # Bandwidth of square waves
T = 2*np.pi  # Period
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f1(t, tau, T, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (tau, T, phi)
a, b = fourier_coeffs(f1, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f1(t, tau, T, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T, fs=fs),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.1)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 2) Sawtooth wave

The generalized definition of a periodic sawtooth wave with $T$ period and $\phi$ phase shift is

$$
    f(t)
    =
    2 \left( \frac{t}{T} - \left\lfloor \frac{1}{2} + \frac{t}{T} \right\rfloor \right),
$$

where $\lfloor \dots \rfloor$ is the floor function.

In [ ]:
def f2(t, T, phi=0):
    '''
    Generate a sawtooth wave signal with a period of :math:`T`, an
    amplitude of ::math:`1` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T/2)%T - T/2
    x = 2 * (t / T - np.floor(1/2 + t/T))

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f2(t, T, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, phi)
a, b = fourier_coeffs(f2, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f2(t, T, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T, fs=fs),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 3) "Periodic" absolute value function $-$ Triangle wave

In this section we consider a periodic, triangle wave-like function with a phase shift defined by

$$
    f(t)
    =
    \left| (t - \phi) \bmod T \right|
    \qquad
    t \in \left[ -\pi, \pi \right].
$$

In [ ]:
def f3(t, T, phi=0):
    '''
    Generate a triangle wave signal with a period of :math:`T`, an
    amplitude of ::math:`T/2` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T/2)%T - T/2
    x = np.abs(t)

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f3(t, T, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, phi)
a, b = fourier_coeffs(f3, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f3(t, T, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T, fs=fs),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 4) Shifted sawtooth

In this section we consider a sawtooth-like periodic function with a phase shift defined by

$$
    f(t)
    =
    1 +  \left( t - \phi \right) \bmod T
    \qquad
    t \in \left[ -\pi, \pi \right].
$$

In [ ]:
def f4(t, T, phi=0):
    '''
    Generate a sawtooth-like signal with a period of :math:`T`, an
    amplitude of ::math:`T/2` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T/2)%T - T/2
    x = 1 + t

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f4(t, T, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, phi)
a, b = fourier_coeffs(f4, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f4(t, T, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T, fs=fs),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 5) Periodic parabolae

In this section we consider a periodic function with a phase shift defined by

$$
    f(t)
    =
    \left[ \left( t - \phi \right) \bmod T \right]^{2}
    \qquad
    t \in \left[ -\pi, \pi \right].
$$

In [ ]:
def f5(t, T, phi=0):
    '''
    Generate a signal with a period of :math:`T`, an amplitude of
    ::math:`T^{2}/4` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T/2)%T - T/2
    x = t*t

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f5(t, T, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, phi)
a, b = fourier_coeffs(f5, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f5(t, T, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T, fs=fs),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 6) Inverted periodic parabolae

In this section we consider a periodic function with a phase shift defined by

$$
    f(t)
    =
    \pi^2 - \left[ \left( t - \phi \right) \bmod T \right]^{2}
    \qquad
    t \in \left[ -\pi, \pi \right].
$$

In [ ]:
def f6(t, T, phi=0):
    '''
    Generate a signal with a period of :math:`T`, an amplitude of
    ::math:`T^{2}/4` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T/2)%T - T/2
    x = np.pi*np.pi - t*t

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f6(t, T, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, phi)
a, b = fourier_coeffs(f6, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f6(t, T, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T, fs=fs),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()

#### 7) Inverted "smooth" sawtooth

In this section we consider a periodic function with a phase shift defined by

$$
    f(t_{c})
    =
    \left( t_{c} - 1 \right) \left( t_{c} - 3\right),
    \qquad
    t_{c} = \left( t - \phi \right) \bmod T,
    \qquad
    t \in \left[ -\pi, \pi \right].
$$

In [ ]:
def f7(t, T, phi=0):
    '''
    Generate a signal with a period of :math:`T`, an amplitude of
    ::math:`T^{2}/4` and a phase shift of :math:`\phi`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    T : float
        Period of the output signal
    phi : float
        Phase shift of the output signal

    Returns
    -------
    x : numpy.array
        The evaluated signal at `t` time points
    '''
    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    t = (t - phi + T/2)%T - T/2
    x = (t - 1) * (t - 3)

    if FCHECK:
        return x[0]
    return x

In [ ]:
# Function-dependent extra parameters
T = 2*np.pi  # Period
phi = 0      # Phase shift
# Sampling locations corresponding to the evaluated signal
t = np.linspace(-T, T, endpoint=False, num=fs)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f7(t, T, phi), color='0.3', lw=4)
ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)

plt.show()

In [ ]:
# Calculate Fourier coefficients
args = (T, phi)
a, b = fourier_coeffs(f7, T, N, 0, *args)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, f7(t, T, phi), color='red', lw=4, alpha=1.0)
for ni in [30, 7, 3]:
    ax.plot(t, fourier_sin(t, a, b, N=ni, T=T, fs=fs),
            label=f'{ni}th order', lw=2, alpha=0.9)

ax.set_xlabel('Time', fontsize=14)
ax.set_ylabel('Amplitude', fontsize=14)
ax.set_ylim(None, ax.get_ylim()[1]*1.15)

ax.legend(loc='upper center', ncol=3)

plt.show()